In [22]:
# List all files in Colab
!ls -lha /content/

total 16K
drwxr-xr-x 1 root root 4.0K Nov 20 14:30 .
drwxr-xr-x 1 root root 4.0K Dec  8 15:39 ..
drwxr-xr-x 4 root root 4.0K Nov 20 14:30 .config
drwxr-xr-x 1 root root 4.0K Nov 20 14:30 sample_data


In [7]:
# Check current directory
import os
print(f"Current directory: {os.getcwd()}")
print(f"Files: {os.listdir('.')}")

Current directory: /content
Files: ['.config', 'sample_data']


In [1]:
# Check GPU availability
!nvidia-smi

Mon Dec  8 15:47:32 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   38C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
# Install dependencies
!pip install onnxruntime-gpu numpy -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.5/300.5 MB 4.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.5/300.5 MB 4.3 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 9.0 MB/s eta 0:00:00


In [24]:
# Set model path
import os

MODEL_PATH = "diarization_transformer_optimized.onnx"

# Check if exists
if os.path.exists(MODEL_PATH):
    size_kb = os.path.getsize(MODEL_PATH) / 1024
    print(f"✅ Model found: {size_kb:.1f} KB")
else:
    # List all files to debug
    print(f"❌ Model not found at: {MODEL_PATH}")
    print(f"Current directory: {os.getcwd()}")
    print(f"All files: {os.listdir('.')}")
    
    # Try to find any .onnx files
    import glob
    onnx_files = glob.glob("**/*.onnx", recursive=True)
    if onnx_files:
        print(f"Found .onnx files: {onnx_files}")
        MODEL_PATH = onnx_files[0]
        print(f"Using: {MODEL_PATH}")
    else:
        print("\n💡 Please use Colab's file browser (folder icon on left) to upload:")
        print("   voiceflow-ml/notebooks/diarization_transformer_optimized.onnx")

❌ Model not found at: diarization_transformer_optimized.onnx
Current directory: /content
All files: ['.config', 'sample_data']

💡 Please use Colab's file browser (folder icon on left) to upload:
   voiceflow-ml/notebooks/diarization_transformer_optimized.onnx


In [5]:
import onnxruntime as ort
import numpy as np
import time
from typing import List

def benchmark_inference(model_path: str, provider: str, iterations: int = 200) -> dict:
    """
    Benchmark ONNX inference with specified provider.
    
    Args:
        model_path: Path to ONNX model file
        provider: 'CUDAExecutionProvider' or 'CPUExecutionProvider'
        iterations: Number of inference runs
    """
    print(f"\n🚀 Benchmarking with {provider}...")
    
    # Create session
    session = ort.InferenceSession(
        model_path,
        providers=[provider]
    )
    
    # Get input shape
    input_name = session.get_inputs()[0].name
    input_shape = list(session.get_inputs()[0].shape)
    
    print(f"Model input: {input_name}, shape: {input_shape}")
    
    # Generate random input (adjust shape if needed)
    # Assuming input is (batch_size, sequence_length, features)
    if input_shape[0] is None or isinstance(input_shape[0], str):
        input_shape[0] = 1  # Batch size
    
    test_input = np.random.randn(*input_shape).astype(np.float32)
    
    # Warmup
    print("Warming up...")
    for _ in range(10):
        session.run(None, {input_name: test_input})
    
    # Benchmark
    print(f"Running {iterations} iterations...")
    latencies = []
    
    for i in range(iterations):
        start = time.perf_counter()
        session.run(None, {input_name: test_input})
        latency_ms = (time.perf_counter() - start) * 1000
        latencies.append(latency_ms)
        
        if (i + 1) % 50 == 0:
            print(f"  Progress: {i + 1}/{iterations}")
    
    # Calculate statistics
    latencies_sorted = sorted(latencies)
    results = {
        "provider": provider,
        "iterations": iterations,
        "min_ms": min(latencies),
        "max_ms": max(latencies),
        "mean_ms": np.mean(latencies),
        "median_ms": np.median(latencies),
        "p50_ms": latencies_sorted[len(latencies) // 2],
        "p95_ms": latencies_sorted[int(len(latencies) * 0.95)],
        "p99_ms": latencies_sorted[int(len(latencies) * 0.99)],
        "throughput_rps": 1000 / np.mean(latencies)
    }
    
    return results

def print_results(results: dict):
    """Pretty print benchmark results."""
    print(f"\n{'='*60}")
    print(f"Provider: {results['provider']}")
    print(f"Iterations: {results['iterations']}")
    print(f"{'='*60}")
    print(f"Min:        {results['min_ms']:.2f} ms")
    print(f"Mean:       {results['mean_ms']:.2f} ms")
    print(f"Median:     {results['median_ms']:.2f} ms")
    print(f"P50:        {results['p50_ms']:.2f} ms")
    print(f"P95:        {results['p95_ms']:.2f} ms")
    print(f"P99:        {results['p99_ms']:.2f} ms ⭐")
    print(f"Max:        {results['max_ms']:.2f} ms")
    print(f"Throughput: {results['throughput_rps']:.1f} req/s")
    print(f"{'='*60}\n")

In [17]:
# Run GPU benchmark
gpu_results = benchmark_inference(MODEL_PATH, 'CUDAExecutionProvider', iterations=200)
print_results(gpu_results)


🚀 Benchmarking with CUDAExecutionProvider...


NoSuchFile: [ONNXRuntimeError] : 3 : NO_SUCHFILE : Load model from diarization_transformer_optimized.onnx failed:Load model diarization_transformer_optimized.onnx failed. File doesn't exist

In [ ]:
# Run CPU benchmark for comparison
cpu_results = benchmark_inference(MODEL_PATH, 'CPUExecutionProvider', iterations=200)
print_results(cpu_results)

In [ ]:
# Comparison
print("\n🎯 GPU vs CPU Comparison")
print(f"{'='*60}")
speedup = cpu_results['mean_ms'] / gpu_results['mean_ms']
print(f"GPU P99:     {gpu_results['p99_ms']:.2f} ms")
print(f"CPU P99:     {cpu_results['p99_ms']:.2f} ms")
print(f"Speedup:     {speedup:.1f}x faster on GPU")
print(f"GPU Target:  < 10 ms P99 {'✅' if gpu_results['p99_ms'] < 10 else '❌'}")
print(f"{'='*60}")

# Save results
import json
results_file = 'benchmark_results.json'
with open(results_file, 'w') as f:
    json.dump({
        'gpu': gpu_results,
        'cpu': cpu_results,
        'speedup': speedup
    }, f, indent=2)

print(f"\n💾 Results saved to {results_file}")

# Download results (Colab only)
try:
    from google.colab import files
    files.download(results_file)
    print("📥 Download started!")
except ImportError:
    print(f"💡 Running locally - results saved to: {os.path.abspath(results_file)}")